In [31]:
import numpy as np
import pandas as pd
import cv2
import pytesseract
from glob import glob
import spacy 
import re
import string




In [32]:

def cleanText(txt):
    whitespace = string.whitespace
    punctuation = '!"#$%&\'-()*+:;<=>?[\\]^_`{|}~'
    tableWhitespace = str.maketrans('','',whitespace)
    tablePunctuation = str.maketrans('','',punctuation)
    text = str(txt)
    text = text.lower()
    removeWhitespace = text.translate(tableWhitespace)
    removePunctuation = removeWhitespace.translate(tablePunctuation)

    return str(removePunctuation)

In [33]:
model_ner = spacy.load('./output/model-best/')

In [34]:
image = cv2.imread('./data/011.jpeg')
# cv2.imshow('bill',image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

tess_data = pytesseract.image_to_data(image)
tess_list = list(map(lambda x: x.split('\t'), tess_data.split('\n')))
df = pd.DataFrame(tess_list[1:], columns= tess_list[0])
df.dropna(inplace=True)
df['text'] =df['text'].apply(cleanText)

df_clean = df.query('text != "" ')
content = " ".join([w for w in df_clean['text']])
print(content)

doc = model_ner(content)

invoice no 68688408 date of issue seller torres and sons 4052 brown roads haysstad, nj 06418 tax id 953842239 iban gb84xsno58142814072766 items no. description qty tks nike 747998401 team hustle 3,00 d7 kids blue mid top basketball shoes size 4y us 2. jordan nike kid air 1 retro high 1,00 black gray elephant size 7y 838850013 summary vat 10 total um each each 03/20/2016 client dennis, levy and bowen 523 leslie manors lake colleenville, ct 12053 tax id 904829186 net price net worth vat 10,99 32,97 10 50,00 50,00 10 net worth vat 82,97 8,30 82,97 8,30 gross worth 36,27 55,00 gross worth 91,27 91,27


In [35]:
from spacy import displacy

In [36]:
# displacy.serve(doc,style='ent')

In [37]:
displacy.render(doc,style='ent')

In [38]:
docjson = doc.to_json()
docjson.keys()

dict_keys(['text', 'ents', 'tokens'])

In [39]:
doc_text = docjson['text']

In [40]:
df_tokens = pd.DataFrame(docjson['tokens'])
df_tokens.head()

,id,start,end
0,0,0,7
1,1,8,10
2,2,11,19
3,3,20,24
4,4,25,27


In [46]:
df_tokens = pd.DataFrame(docjson['tokens'])
df_tokens['token'] = df_tokens[['start','end']].apply(lambda x:doc_text[x['start']:x['end']], axis = 1)

df_tokens.head(10)

,id,start,end,token
0,0,0,7,invoice
1,1,8,10,no
2,2,11,19,68688408
3,3,20,24,date
4,4,25,27,of
5,5,28,33,issue
6,6,34,40,seller
7,7,41,47,torres
8,8,48,51,and
9,9,52,56,sons


In [49]:
right_table = pd.DataFrame(docjson['ents'])[['start','label']]
df_tokens = pd.merge(df_tokens,right_table, how = "left", on='start')

In [50]:
df_tokens.head(10)

,id,start,end,token,label
0,0,0,7,invoice,NaN
1,1,8,10,no,NaN
2,2,11,19,68688408,B-ID
3,3,20,24,date,NaN
4,4,25,27,of,NaN
5,5,28,33,issue,NaN
6,6,34,40,seller,NaN
7,7,41,47,torres,B-SN
8,8,48,51,and,I-SN
9,9,52,56,sons,I-SN


In [52]:
df_tokens.fillna('O', inplace=True)
df_tokens.head(10)

,id,start,end,token,label
0,0,0,7,invoice,O
1,1,8,10,no,O
2,2,11,19,68688408,B-ID
3,3,20,24,date,O
4,4,25,27,of,O
5,5,28,33,issue,O
6,6,34,40,seller,O
7,7,41,47,torres,B-SN
8,8,48,51,and,I-SN
9,9,52,56,sons,I-SN
